## 练习：星巴克
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### 背景信息

此练习将提供一个数据集，星巴克原先使用该数据集作为面试题。这道练习的数据包含 120,000 个数据点，按照 2:1 的比例划分为训练文件和测试文件。数据模拟的实验测试了一项广告宣传活动，看看该宣传活动能否吸引更多客户购买定价为 10 美元的特定产品。由于公司分发每份宣传资料的成本为 0.15 美元，所以宣传资料最好仅面向最相关的人群。每个数据点都有一列表示是否向某个人发送了产品宣传资料，另一列表示此人最终是否购买了该产品。每个人还有另外 7 个相关特征，表示为 V1-V7。

#### 优化策略

你的任务是通过训练数据了解 V1-V7 存在什么规律表明应该向用户分发宣传资料。具体而言，你的目标是最大化以下指标：

* **增量响应率 (IRR)** 

IRR 表示与没有收到宣传资料相比，因为推广活动而购买产品的客户增加了多少。从数学角度来说，IRR 等于推广小组的购买者人数与购买者小组客户总数的比例 (_treatment_) 减去非推广小组的购买者人数与非推广小组的客户总数的比例 (_control_)。

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **净增量收入 (NIR)**

NIR 表示分发宣传资料后获得（丢失）了多少收入。从数学角度来讲，NIR 等于收到宣传资料的购买者总人数的 10 倍减去分发的宣传资料份数的 0.15 倍，再减去没有收到宣传资料的购买者人数的 10 倍。

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

要详细了解星巴克提供给应聘者的数据集，请参阅[此处的说明](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM)。

下面是训练数据。研究数据和不同的优化策略。

#### 如何测试你的策略？

如果你想到了优化策略，请完成要传递给 `test_results` 函数的 `promotion_strategy` 函数。  
根据以往的数据，我们知道有四种可能的结果：

实际推广客户与预测推广客户表格：  

<table>
<tr><th></th><th colspan = '2'>实际</th></tr>
<tr><th>预测</th><th>是</th><th>否</th></tr>
<tr><th>是</th><td>I</td><td>II</td></tr>
<tr><th>否</th><td>III</td><td>IV</td></tr>
</table>

我们仅针对预测应该包含推广活动的个人比较了指标，即第一象限和第二象限。由于收到宣传资料的第一组客户（在训练集中）是随机收到的，因此第一象限和第二象限的参与者人数应该大致相同。  

比较第一象限与第二象限可以知道宣传策略未来效果如何。 

首先阅读以下数据。看看每个变量或变量组合与推广活动对购买率有何影响。你想到谁应该接收宣传资料的策略后，请使用在最后的 `test_results` 函数中使用的测试数据集测试你的策略。

# 1. 导入数据和初步查看数据

In [2]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [8]:
# 查看数据大小
train_data.shape

(84534, 10)

数据包含84534行10列，每列的含义如下：
1. ID: 用户ID
2. Promotion: 是否获得选出资料
3. purchase：是否交易
4. V1-V7: 用户的7个特征

In [9]:
train_data.describe()

,ID,purchase,V1,V2,V3,V4,V5,V6,V7
count,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000,84534.000000
mean,62970.972413,0.012303,1.500662,29.973600,0.000190,1.679608,2.327643,2.502898,1.701694
std,36418.440539,0.110234,0.868234,5.010626,1.000485,0.466630,0.841167,1.117349,0.457517
min,1.000000,0.000000,0.000000,7.104007,-1.684550,1.000000,1.000000,1.000000,1.000000
25%,31467.250000,0.000000,1.000000,26.591501,-0.905350,1.000000,2.000000,2.000000,1.000000
50%,62827.500000,0.000000,2.000000,29.979744,-0.039572,2.000000,2.000000,3.000000,2.000000
75%,94438.750000,0.000000,2.000000,33.344593,0.826206,2.000000,3.000000,4.000000,2.000000
max,126184.000000,1.000000,3.000000,50.375913,1.691984,2.000000,4.000000,4.000000,2.000000


In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84534 entries, 0 to 84533
Data columns (total 10 columns):
ID           84534 non-null int64
Promotion    84534 non-null object
purchase     84534 non-null int64
V1           84534 non-null int64
V2           84534 non-null float64
V3           84534 non-null float64
V4           84534 non-null int64
V5           84534 non-null int64
V6           84534 non-null int64
V7           84534 non-null int64
dtypes: float64(2), int64(7), object(1)
memory usage: 6.4+ MB


In [13]:
train_data['Promotion'].value_counts()

Yes    42364
No     42170
Name: Promotion, dtype: int64

In [14]:
train_data['purchase'].value_counts()

0    83494
1     1040
Name: purchase, dtype: int64

In [15]:
train_data['V1'].value_counts()

1    31631
2    31608
3    10670
0    10625
Name: V1, dtype: int64

In [19]:
train_data['V2'].describe()

count    84534.000000
mean        29.973600
std          5.010626
min          7.104007
25%         26.591501
50%         29.979744
75%         33.344593
max         50.375913
Name: V2, dtype: float64

In [21]:
train_data['V3'].describe()

count    84534.000000
mean         0.000190
std          1.000485
min         -1.684550
25%         -0.905350
50%         -0.039572
75%          0.826206
max          1.691984
Name: V3, dtype: float64

In [23]:
train_data['V4'].value_counts()

2    57450
1    27084
Name: V4, dtype: int64

In [26]:
train_data['V5'].value_counts()

3    32743
2    31196
1    15412
4     5183
Name: V5, dtype: int64

In [28]:
train_data['V6'].value_counts()

3    21186
4    21176
2    21146
1    21026
Name: V6, dtype: int64

In [30]:
train_data['V7'].value_counts()

2    59317
1    25217
Name: V7, dtype: int64

可以看到各列数据没有缺失值，均为完整的数据。
- ID的范围1~126184的整数，表示用户的ID
- Promotion为只有Yes/No的类别型变量，表示是否收到推广信息，Yes的数量为42364，No的数量为42170
- purchase为只有0/1的类别型变量，表示是否购买咖啡，0表示没有购买数量为83494，1表示购买了咖啡数量为1040
- V1为用户的特征，有0/1/2/3这4种取值，属于类别型数据
- V2为用户的特征，应该是属于数值型数据
- V3为用户的特征，其均值为0，方差接近1，可能是经过标准化的数据，为数值型数据
- V4为用户的特征，有1/2这2种取值，属于类别型数据
- V5为用户的特征，有1/2/3/4这4种取值，属于类别型数据
- V6为用户的特征，有1/2/3/4这4种取值，属于类别型数据
- V7为用户的特征，有1/2这2种取值，属于类别型数据

# 2. 不变指标
是否收到推广信息的用户数量为不变指标，所有用户收到推广信息的概率应该是均等的，即概率为0.5

In [66]:
# 不变指标的Z参数检验
n_ctrl = (train_data['Promotion'] == 'No').sum()
n_obs  = train_data.shape[0]
p = 0.5 
sd = np.sqrt(n_obs * p * (1 - p))
z = ((n_ctrl - 0.5) - n_obs * p) / sd
print('Z-score: {:.4f}'.format(z))
p_value = 2 * sp.stats.norm.cdf(z)
print('Z检验p-value: {:.4f}'.format(p_value))

Z-score: -0.6707
Z检验p-value: 0.5024


# 3. 评估指标
推广对促进购买率是否有统计显著性效果，考察的指标为：
- 购买率：购买量/用户数量

In [69]:
# 购买率的Z参数检验
n_treat = n_obs - n_ctrl
# 实验组的购买数量
k_treat = train_data[train_data['Promotion'] == 'Yes']['purchase'].sum()
# 对照组的购买数量
k_ctrl = train_data[train_data['Promotion'] == 'No']['purchase'].sum()
# 实验组和对照组的购买率
p_treat = k_treat / n_treat
p_ctrl = k_ctrl / n_ctrl
# 计算Z参数
sd = np.sqrt(p_treat * (1 - p_treat) / n_treat + p_ctrl * (1 - p_ctrl) / n_ctrl)
z = (p_treat - p_ctrl) / sd
print('Z-score: {:.4f}'.format(z))
p_value = 1 - sp.stats.norm.cdf(z)
print('Z检验p-value: {}'.format(p_value))

Z-score: 12.4908
Z检验p-value: 0.0


In [70]:
sd

0.0007569189294319118

推广对提升用户的购买率的提升具有统计显著性，说明推广对提升购买率具有效果，可以采用此种方法进行推广。

接下来，需要训练一个推广策略，来提升推广的效率，具体在指标的体现上就是，提升IRR和NIR，相当于是精准的投放推广信息，毕竟每一份推广都是有成本的。

可以使用用户的特征作为训练数据的特征，对于标签选择Promotion和purchase的“与”运算结果。

In [ ]:
# Cells for you to work and document as necessary - 
# definitely feel free to add more cells as you need

In [ ]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    
    
    
    return promotion

In [ ]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)